In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu transformers[torch] #torch==1.9.1

In [ ]:
pip install transformers[torch]

In [ ]:
# ! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
# ! protoc --python_out=. sentencepiece_model.proto
# import sentencepiece_model_pb2

--2022-11-23 05:24:44--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13451 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

sentencepiece_model 100%[===================>]  13.14K  --.-KB/s    in 0s      

2022-11-23 05:24:45 (66.3 MB/s) - ‘sentencepiece_model.proto’ saved [13451/13451]



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


# Fine-tuning a model on a translation task

In [ ]:
model_checkpoint = 'Elaine/m2m100_pruned_ta'

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("sacrebleu")

<ipython-input-5-29406150f423>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
data_files = {"train": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/small_MCSW_Bitext_shuf.json"}
raw_datasets = load_dataset("json", data_files=data_files)
raw_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ed0d01a9ae74be14/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 100000
    })
})

In [ ]:
raw_datasets['train']['translation'][:5]

[{'en': 'So you need to know that, while you and your team have been in the temporal zone, I made a choice that affected the timeline.',
  'mcsw': 'So you need to know, semasa kau dan pasukan kau . . . berada di Zon Sementara aku membuat keputusan yang membawa kesan pada timeline .',
  'lang_switch': ['en']},
 {'en': 'But in the 1970s, some countries caught up.',
  'mcsw': '到了 70年代， some countries he has come,',
  'lang_switch': ['en', 'en']},
 {'en': 'I was an accident. A random convergence of technology. - You are unique.',
  'mcsw': 'I was an accident . A random convergence of technology a : Awak memang unik.',
  'lang_switch': ['ms']},
 {'en': 'In recent years, the Government of the United States has attempted to mislead international public opinion by means of a supposed relaxation of the regulations governing the embargo which would allegedly allow the sale to Cuba of food and medicines.',
  'mcsw': '23 . 近年来美国政府想要借着所谓放宽有关禁运的条例从而准许向古巴出售粮食gung yoek mat lei .lou dim 国际舆论',
  'lang_

In [ ]:
data_files = {"train": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/en_augment_shuf.json",
              "validation": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/valid_en.json",
              "test": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/test_en.json"}
raw_datasets = load_dataset("json", data_files=data_files)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
raw_datasets["test"][0]

{'translation': {'en': 'ROC got fucked by commie, ran to Taiwan in 1949 and ruled as a damn dictatorship',
  'mcsw': "ROC kena knn'ed by commie cabut to Taiwan in 1949 and ruled as a knn dictatorship",
  'lang_switch': []}}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["test"])

,translation
0,"{'en': 'just nice 10% drop, increase bet~~', 'mcsw': 'ngam ngam 10% drop, 加码 ~ ~', 'lang_switch': []}"
1,"{'en': 'DLCW's best. Can be appointed as an icon of sports and Malaysian unity. Nice to read Dato's post', 'mcsw': 'Terbaik DLCW.Boleh diangakat jadi ikon sukan dan perpaduan Malaysia.Syok baca posting Dato', 'lang_switch': []}"
2,"{'en': 'Best read Dato' story <emoji>.. Happy National Day legendary idol.', 'mcsw': 'Best baca story Dato’ <emoji>.. Selamat Hari Kebangsaan legend idol.', 'lang_switch': []}"
3,"{'en': 'They hear Peninsular's voice. Guarantee price increases 100%.', 'mcsw': 'They hear semenanjung voice. gurantee price increase 100%.', 'lang_switch': []}"
4,"{'en': 'Datuk's comment did not disappoint…....', 'mcsw': 'comment Datuk tak mengecewakan la.......', 'lang_switch': []}"


## Preprocessing the data

### Modifying tokenizer for extra language 'MCSW'

Use this if finetuning using src_lang = 'mcsw'

In [ ]:
from transformers import M2M100Tokenizer

# https://github.com/TartuNLP/m2m-100-finetune/blob/main/tok.py
def _update_tok_lang_dicts(tokenizer):
  tokenizer.id_to_lang_token = dict(list(tokenizer.id_to_lang_token.items()) + list(tokenizer.added_tokens_decoder.items()))
  tokenizer.lang_token_to_id = dict(list(tokenizer.lang_token_to_id.items()) + list(tokenizer.added_tokens_encoder.items()))
  tokenizer.lang_code_to_token = { k.replace("_", ""): k for k in tokenizer.additional_special_tokens }
  tokenizer.lang_code_to_id = { k.replace("_", ""): v for k, v in tokenizer.lang_token_to_id.items() }

def get_extended_m2m100_tokenizer(pretrained_model, new_lang_list):
  lang_toks = ["__" + lang + "__" for lang in new_lang_list]
  tokenizer = M2M100Tokenizer.from_pretrained(pretrained_model, additional_special_tokens = lang_toks)
  _update_tok_lang_dicts(tokenizer)
  return tokenizer

tokenizer = get_extended_m2m100_tokenizer(model_checkpoint, ['mcsw'])
tokenizer.src_lang = "mcsw"
tokenizer.tgt_lang = "en"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Use this if finetuning NOT using src_lang='mcsw'; Replace src_lang with respective lang e.g. 'en', 'zh', 'ms', 'ta'

In [ ]:
from transformers import M2M100Tokenizer
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint)
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "en"
len(tokenizer)

44969

In [ ]:
len(tokenizer)

44971

In [ ]:
max_input_length = 128
max_target_length = 128
prefix = ""
# source_lang = 'mcsw'
source_lang = tokenizer.src_lang
target_lang = 'en'

def preprocess_function(examples):
    # inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    inputs = [prefix + ex['mcsw'] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
raw_datasets['train'][:3]

{'translation': [{'en': 'The actions referred to in paragraph 2 shall be carried out, as appropriate, in close consultation with other relevant national authorities and without prejudice to their specific competencies.',
   'mcsw': 'The actions referred to in paragraph 2 naklah dilaksann, jika berkenaan, in close consultation with other relevant national authorities and without prejudice kepe their specific competencies',
   'lang_switch': ['ms', 'ms']},
  {'en': 'Strategic keys The competitiveness of the Basque audiovisual sector is determined by a series of key factors and characteristics which will increase its strategic position in the immediate future.',
   'mcsw': 'Strategic keys The competitiveness of the Basque audiovisual sector is determined by a series of key factors dan ciri-ciri yang akan increase its strategic position in the immediate future .',
   'lang_switch': ['ms']},
  {'en': '(i) Where the educational institution provides board, the amount shall be 75 per cent of t

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[44887, 1365, 40463, 4235, 1455, 126, 28, 27752, 36764, 165, 1229, 3581, 2991, 1529, 872, 4, 5521, 17487, 4, 28, 41051, 13426, 847, 5929, 29240, 25066, 15305, 40162, 12428, 890, 41782, 659, 8269, 840, 5211, 182, 28702, 25725, 21963, 5684, 2], [44887, 22295, 346, 484, 713, 1365, 8885, 9463, 48, 1184, 415, 1028, 4244, 1115, 32022, 34539, 8408, 115, 8934, 237, 656, 8, 29115, 415, 14168, 42820, 161, 28214, 7, 140, 944, 148, 1180, 44133, 8016, 8801, 346, 22528, 28, 1028, 41483, 393, 41065, 233, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[44887, 1365, 40463, 4235, 1455, 126, 28, 27752, 36764, 165, 40973, 618, 1500, 10965, 6145, 4, 279, 44586, 4, 28, 41051, 13426, 847, 5929, 29240, 25066, 15305, 40162, 12428, 890, 41782, 659, 8

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## Fine-tuning the model

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Postprocess & evaluation function.

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model.config.vocab_size

44971

# Fine-tuning

## MCSW Bitext

We can now finetune our model by just calling the `train` method:

In [ ]:
for i, param in enumerate(model.parameters()):
  print(i)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'halfmcswbitext'
args = Seq2SeqTrainingArguments(
    f"{model_name}-freeze100-finetuned-{source_lang}-to-{target_lang}-{datatype}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
    # resume_from_checkpoint=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 10 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,1.432700,1.820922,29.567000,27.648300
5000,1.301800,1.786061,32.061500,27.851700
7500,1.257400,1.791163,30.627200,26.931000
10000,1.228700,1.770336,31.544400,26.813800
12500,1.212700,1.762387,32.062500,28.010300


Upload file runs/Apr16_09-16-34_ab5768a5e7d3/events.out.tfevents.1681636608.ab5768a5e7d3.152.0: 100%|#########…

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext
   f12a4da..4275a4e  main -> main

   f12a4da..4275a4e  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext
   4275a4e..97f9f85  main -> main

   4275a4e..97f9f85  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext/commit/4275a4e4e33c3537eb97b27404112edd3d84ea6f'

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'halfmcswbitext2'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext2",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
    # resume_from_checkpoint=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext2 into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,1.216100,1.830811,30.847100,28.237900
5000,1.209100,1.795851,30.815500,27.517200
7500,1.172100,1.795721,30.062300,27.662100
10000,1.165800,1.776649,28.309200,27.493100
12500,1.150400,1.764353,31.051900,27.537900
15000,1.137300,1.761467,31.284800,27.144800


To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext2
   1e53aa2..54d4571  main -> main

   1e53aa2..54d4571  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext2
   54d4571..65de3cc  main -> main

   54d4571..65de3cc  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-mcsw-to-en-halfmcswbitext2/commit/54d4571ce4dceabf5a2da459cb8de516b3ec9e25'

In [ ]:
import datetime
import pandas as pd
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = 'halfmcswbitext2'
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mcswbitext_training_loss2.csv', index=False)

eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mcswbitext_eval_score.csv', 'a') as f:
    f.write(f"{datatype},{eval_score}\n")

31.2848


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer

# Load model and tokenizer
# model = AutoModelWithLMHead.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Preprocess input text
input_text = "I'm looking for a good restaurant in Paris"
input_ids = torch.tensor(tokenizer.encode(input_text, return_tensors='pt')).unsqueeze(0)  # Batch size 1

# Generate predictions
 # The first element of the output tuple is the output from the model
trainer.predict(torch.Tensor(tokenized_datasets['train'][1]['input_ids']))

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4
Configuration saved in m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/config.json
Model weights saved in m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/pytorch_model.bin
tokenizer config file saved in m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/tokenizer_config.json
Special tokens file saved in m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/special_tokens_map.json
added tokens file saved in m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/added_tokens.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/1.49G [00:00<?, ?B/s]

Upload file runs/Jan17_15-47-58_9e059be7a6f1/events.out.tfevents.1673970499.9e059be7a6f1.443.0:  26%|##5      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4
   77da31a..6bc880c  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4
   77da31a..6bc880c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4
   6bc880c..47e5b37  main -> main

   6bc880c..47e5b37  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en-mcswbitext-batch4/commit/6bc880cf47ff7eba2a3b1ad172532e1f805d89c7'

## MonoBitext

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'halfmcswbitext'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
    # resume_from_checkpoint=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 10 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,1.414500,1.906749,29.547900,28.331000
5000,1.253000,1.858247,30.919800,27.320700
7500,1.210800,1.838264,30.943000,27.003400
10000,1.185200,1.826459,30.056400,26.655200
12500,1.171100,1.818631,30.982800,27.379300


To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext
   2ae20b2..a13d9b4  main -> main

   2ae20b2..a13d9b4  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext
   a13d9b4..450fb00  main -> main

   a13d9b4..450fb00  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext/commit/a13d9b480cd38c7682ecbce270144b1dec58494b'

In [ ]:
import datetime
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = 'halfmcswbitext'
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mono+mcswbitext_training_loss.csv', index=False)

In [ ]:
# Save the BLEU evaluation to the CSV file using the pandas library
eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mono+mcswbitext_eval_score.csv', 'a') as f:
    f.write(f"freeze100,{eval_score}\n")

30.9828


In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'halfmcswbitext2'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext2",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
    # resume_from_checkpoint=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 10 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext2 into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,1.172600,1.878489,28.854200,27.917200
5000,1.167200,1.830561,30.902600,26.862100
7500,1.134200,1.840900,28.118700,28.048300
10000,1.129000,1.800683,30.551800,27.141400
12500,1.115600,1.800508,30.210700,27.610300
15000,1.102800,1.798648,29.908200,26.893100


Upload file runs/Apr22_10-44-48_b19eda7afa74/events.out.tfevents.1682160293.b19eda7afa74.232.0: 100%|#########…

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext2
   49b6770..d73b2b6  main -> main

   49b6770..d73b2b6  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext2
   d73b2b6..f25c2e9  main -> main

   d73b2b6..f25c2e9  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-monobitext-finetuned-halfmcswbitext2/commit/d73b2b62a6ceaa1e6832fbe13332c4f885228cc2'

In [ ]:
import datetime
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = 'halfmcswbitext'
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mono+mcswbitext_training_loss.csv', index=False)

# Save the BLEU evaluation to the CSV file using the pandas library
eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open('/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MCSW_Bitext_Data/mono+mcswbitext_eval_score.csv', 'a') as f:
    f.write(f"freeze100,{eval_score}\n")

29.9082


## MCSW - Use lang instead of MCSW as src_lang

MS

In [ ]:
#@title
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'ms_augmented'
args = Seq2SeqTrainingArguments(
    f"{model_name}-freeze100-{datatype}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 1500,
    save_steps = 1500,
    logging_steps = 1500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
1500,2.168600,1.785526,28.628000,24.803400
3000,1.997200,1.747560,28.349200,24.093100
4500,1.925000,1.755421,26.098000,23.596600
6000,1.876100,1.742011,27.641600,24.762100


Upload file runs/Apr25_06-28-03_491f9227be24/events.out.tfevents.1682404094.491f9227be24.810.0: 100%|#########…

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented
   2ce0f62..302f965  main -> main

   2ce0f62..302f965  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented
   302f965..e66b8a8  main -> main

   302f965..e66b8a8  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented/commit/302f965fe8ef7058c52c5c50ea6f3ad809f02424'

TA

In [ ]:
# @title
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'ta_augmented'
args = Seq2SeqTrainingArguments(
    f"{model_name}-freeze100-{datatype}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 1500,
    save_steps = 1500,
    logging_steps = 1500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-ta_augmented into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
1500,2.805100,2.140045,27.983600,26.724100


To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-ta_augmented
   95e5742..bb0f356  main -> main

   95e5742..bb0f356  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-ta_augmented
   bb0f356..f67be56  main -> main

   bb0f356..f67be56  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-ta_augmented/commit/bb0f356e350db13c8f90beb9081ba73b3b5a88e3'

EN

In [ ]:
# @title
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'en_augmented'
args = Seq2SeqTrainingArguments(
    f"m2m100_pruned_ta-freeze100-{datatype}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 1500,
    save_steps = 1500,
    logging_steps = 1500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-en_augmented into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
1500,0.478600,2.014297,30.030900,25.524100
3000,0.379400,2.039228,29.891200,25.410300
4500,0.355600,2.085531,29.881100,26.193100
6000,0.336000,2.056787,29.936000,25.265500
7500,0.322700,2.054965,30.134500,25.758600
9000,0.315300,2.044195,30.515700,25.420700
10500,0.312500,2.031362,30.441400,24.837900
12000,0.302700,2.027004,30.153200,25.337900
13500,0.301200,2.031487,29.943300,25.175900


Upload file runs/Apr30_03-53-15_7cb1dbf6a13c/events.out.tfevents.1682826806.7cb1dbf6a13c.320.0:   0%|         …

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-en_augmented
   b5d477d..d7ff069  main -> main

   b5d477d..d7ff069  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-en_augmented
   d7ff069..0684ec4  main -> main

   d7ff069..0684ec4  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-en_augmented/commit/d7ff0691568070d0ec00271ab24c704338fb05ed'

ZH

In [ ]:
#@title
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'zh_augmented'
args = Seq2SeqTrainingArguments(
    f"{model_name}-freeze100-{datatype}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 1500,
    save_steps = 1500,
    logging_steps = 1500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False

trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-zh_augmented into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
1500,2.104100,2.234561,19.257000,28.813800
3000,1.998100,2.242157,18.345700,26.920700
4500,1.952500,2.208904,19.776000,28.317200
6000,1.910600,2.227070,18.565300,26.948300
7500,1.894000,2.199870,20.058100,27.003400


To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-zh_augmented
   2ecc3ea..e90b514  main -> main

   2ecc3ea..e90b514  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-zh_augmented
   e90b514..7a14540  main -> main

   e90b514..7a14540  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-ms_augmented-freeze100-zh_augmented/commit/e90b514ac6b3f5993b91d3313378954323f5a332'

# Testing

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def _update_tok_lang_dicts(tokenizer):
  tokenizer.id_to_lang_token = dict(list(tokenizer.id_to_lang_token.items()) + list(tokenizer.added_tokens_decoder.items()))
  tokenizer.lang_token_to_id = dict(list(tokenizer.lang_token_to_id.items()) + list(tokenizer.added_tokens_encoder.items()))
  tokenizer.lang_code_to_token = { k.replace("_", ""): k for k in tokenizer.additional_special_tokens }
  tokenizer.lang_code_to_id = { k.replace("_", ""): v for k, v in tokenizer.lang_token_to_id.items() }

def get_extended_m2m100_tokenizer(pretrained_model, new_lang_list):
  lang_toks = ["__" + lang + "__" for lang in new_lang_list]
  tokenizer = AutoTokenizer.from_pretrained(pretrained_model, additional_special_tokens = lang_toks)
  _update_tok_lang_dicts(tokenizer)
  return tokenizer

new_model = "Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en"
my_model = AutoModelForSeq2SeqLM.from_pretrained(new_model)
my_tokenizer = get_extended_m2m100_tokenizer(new_model, ['mcsw'])
my_tokenizer.src_lang = "mcsw" # 'ms'

In [ ]:
def my_translation(text, model, tokenizer):
  encoded_text = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_text,
                                    forced_bos_token_id=tokenizer.get_lang_id("en"),
                                    max_new_tokens=40)
  print("Ori: {}\nTrans: {}".format(text, tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)))

Model Before Finetune

In [ ]:
my_translation('Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara ',
               model, tokenizer)
my_translation('dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.',
               model, tokenizer)
my_translation('Aku boleh pinjam tentera from rakan aku , Yin Po .',
               model, tokenizer)
my_translation('Planet aku bernama Lorien but the earth is tempat ku sekarang .',
               model, tokenizer)

Ori: Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara 
Trans: ['Letnan, kita have dua the enemy direction. 300 meters di sebelah utara']
Ori: dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.
Trans: ['And now I am sure... you have something that actually belongs to us.']
Ori: Aku boleh pinjam tentera from rakan aku , Yin Po .
Trans: ['I can pinjam tentera from rakan aku, Yin Po.']
Ori: Planet aku bernama Lorien but the earth is tempat ku sekarang .
Trans: ['Planet I am named Lorien but the earth is tempat ku now.']


In [ ]:
my_translation('Comfortable if we tahu dengan siapa kita bercakap .',
               model, tokenizer)
my_translation('Yeah . Orang dewasa coming ke taman... dan tak pulang hidup-hidup ',
               model, tokenizer)

Ori: Comfortable if we tahu dengan siapa kita bercakap .
Trans: ['Comfortable if we know with whom we talk.']
Ori: Yeah . Orang dewasa coming ke taman... dan tak pulang hidup-hidup 
Trans: ['People come to the park and don’t come home live-life']


Model After Finetune

In [ ]:
# mcswbitext, batch size 8
my_translation('Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara ',
               my_model, my_tokenizer)
my_translation('dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.',
               my_model, my_tokenizer)
my_translation('Aku boleh pinjam tentera from rakan aku , Yin Po .',
               my_model, my_tokenizer)
my_translation('Planet aku bernama Lorien but the earth is tempat ku sekarang .',
               my_model, my_tokenizer)
my_translation('Comfortable if we tahu dengan siapa kita bercakap .',
               my_model, my_tokenizer)
my_translation('Yeah. Orang dewasa coming ke taman... dan tak pulang hidup-hidup ',
               my_model, my_tokenizer)

Ori: Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara 
Trans: ['Letnan, we have two enemy directions 300 meters north.']
Ori: dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.
Trans: ["And now I'm sure you have something that actually belongs to us."]
Ori: Aku boleh pinjam tentera from rakan aku , Yin Po .
Trans: ['I can borrow a military from my friend, Yin Po.']
Ori: Planet aku bernama Lorien but the earth is tempat ku sekarang .
Trans: ['My planet is Lorien, but the Earth is where I am now.']
Ori: Comfortable if we tahu dengan siapa kita bercakap .
Trans: ["It's nice if we know with whom we're talking."]
Ori: Yeah. Orang dewasa coming ke taman... dan tak pulang hidup-hidup 
Trans: ["Yeah, adults come to the park... and don't come home alive."]
